## 导入依赖环境

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import time

def set_seed(s=0):
    random.seed(s)
    np.random.seed(s)
    tf.random.set_seed(s)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv('pre_bank.csv')



categorical_features = [
    'job',
    'marital',
    'education',
    'default',
    'housing',
    'loan',
    'contact',
    'month',
    'poutcome'
]
numeric_features = [col for col in df.columns if col not in categorical_features + ['y']]



In [ ]:


import dice_ml
from dice_ml.utils.helpers import DataTransfomer
transformer = DataTransfomer(func='ohe-min-max')

target = df['y']
train_dataset, test_dataset, y_train, y_test = train_test_split(df, 
                                                                target,
                                                                test_size=0.2, 
                                                                random_state=42, 
                                                                stratify=df['y'])



X_train_df = train_dataset.drop('y', axis=1)
X_test_df = test_dataset.drop('y', axis=1)
d = dice_ml.Data(dataframe=df,
                 continuous_features=numeric_features,
                 outcome_name='y')

transformer.feed_data_params(d)

transformer.initialize_transform_func()

X_train = transformer.transform(X_train_df)
X_test = transformer.transform(X_test_df)




In [ ]:
import numpy as np

# If it is a sparse matrix (such as the output after OneHot + normalization), convert to dense first
if hasattr(X_train, "toarray"):
    X_dense = X_train.toarray()
else:
    X_dense = X_train

# 1. Calculate the centroid (mean of each column)
centroid = np.mean(X_dense, axis=0)

# 2. Calculate the Euclidean distance from each sample to the centroid
distances = np.linalg.norm(X_dense - centroid, axis=1)

# 3. Find the index of the closest sample
closest_idx =

5543

## 待解释模型训练以及评估

In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

def build_simple_dnn():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(32, activation='relu', input_shape=(49,)))  # Input 31-dimensional features
    model.add(keras.layers.Dense(2, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

set_seed(1)

model = build_simple_dnn()

# 6️⃣ Train the model
model.fit(X_train, to_categorical(y_train), epochs=10, batch_size=8, verbose=1)
model.save_weights('my_model_weights_bank.h5')  #

Epoch 1/10
1058/1058 [==============================] - 1s 809us/step - loss: 0.5681 - accuracy: 0.6988
Epoch 2/10
1058/1058 [==============================] - 1s 716us/step - loss: 0.4603 - accuracy: 0.7985
Epoch 3/10
1058/1058 [==============================] - 1s 711us/step - loss: 0.4135 - accuracy: 0.8239
Epoch 4/10
1058/1058 [==============================] - 1s 706us/step - loss: 0.3936 - accuracy: 0.8303
Epoch 5/10
1058/1058 [==============================] - 1s 736us/step - loss: 0.3828 - accuracy: 0.8413
Epoch 6/10
1058/1058 [==============================] - 1s 703us/step - loss: 0.3730 - accuracy: 0.8414
Epoch 7/10
1058/1058 [==============================] - 1s 708us/step - loss: 0.3662 - accuracy: 0.8466
Epoch 8/10
1058/1058 [==============================] - 1s 707us/step - loss: 0.3610 - accuracy: 0.8497
Epoch 9/10
1058/1058 [==============================] - 1s 733us/step - loss: 0.3534 - accuracy: 0.8550
Epoch 10/10
1058/1058 [==============================] - 1s 742u

In [ ]:
# Prediction (using X_test as an example)
y_pred_prob = model.predict(X_test)

# Convert probabilities to class labels
y_pred = y_pred_prob.argmax(axis=1)

# Calculate accuracy
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)

67/67 [==============================] - 0s 605us/step
Accuracy: 0.854


In [ ]:
# Prediction (using X_test as an example)
y_pred_prob = model.predict(X_test)

# Convert probabilities to class labels
y_pred = y_pred_prob.argmax(axis=1)

# Calculate accuracy
from sklearn.metrics import accuracy_score

67/67 [==============================] - 0s 611us/step
              precision    recall  f1-score   support

           0     0.8563    0.8507    0.8535      1058
           1     0.8516    0.8573    0.8545      1058

    accuracy                         0.8540      2116
   macro avg     0.8540    0.8540    0.8540      2116
weighted avg     0.8540    0.8540    0.8540      2116

F1 Macro: 0.8540, F1 Micro: 0.8540


In [8]:
import dice_ml

# Using sklearn backend
m = dice_ml.Model(model=model, 
                  backend="TF2", 
                  func="ohe-min-max")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m,
                   method="kdtree")


In [9]:
from dice_ml.model_interfaces.keras_tensorflow_model import KerasTensorFlowModel
def patched_get_output(self, input_tensor, model_score=True, training=False, transform_data=False):
    import tensorflow as tf
    if transform_data or not tf.is_tensor(input_tensor):
        input_tensor = tf.constant(self.transformer.transform(input_tensor).to_numpy(), dtype=tf.float32)
    output = self.model(input_tensor, training=training)
    return output
KerasTensorFlowModel.get_output = patched_get_output

In [10]:
set_seed(0)

start = time.time()
e1_kdtree = exp.generate_counterfactuals(
    X_test_df[:100],
    total_CFs=4,
    desired_class="opposite",
    features_to_vary = X_train_df.columns[X_train_df.columns != 'sex'].tolist()
)
end = time.time()
time_kdtree = (end - start)/100

100%|██████████| 100/100 [12:03<00:00,  7.24s/it]


In [11]:
exp.generate_counterfactuals(
    X_train_df[closest_idx:closest_idx+1],
    total_CFs=4,
    desired_class="opposite",
    features_to_vary = X_train_df.columns[X_train_df.columns != 'sex'].tolist()
).visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:06<00:00,  6.68s/it]

Query instance (original outcome : 0)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,poutcome,y
0,47,management,married,secondary,no,177,yes,no,cellular,14,may,95,3,unknown,0



Diverse Counterfactual set (new outcome: 1.0)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,poutcome,y
1138,28,-,single,-,-,-,no,-,-,10,mar,-,1,-,1
2720,28,student,single,-,-,-,no,-,-,-,jul,-,-,other,1
444,-,admin.,-,-,-,-,-,-,-,23,jul,-,1,success,1
751,61,retired,-,primary,-,-,no,-,-,17,feb,-,1,-,1


### (2)Random

In [12]:

exp = dice_ml.Dice(d, m,
                   method="random")
start = time.time()
e1_random = exp.generate_counterfactuals(
    X_test_df[:100],
    total_CFs=4,
    desired_class="opposite",
    features_to_vary = X_train_df.columns[X_train_df.columns != 'sex'].tolist()
)
end = time.time()
time_random = (end - start)/100

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:55<00:00,  1.79it/s]


In [13]:
exp.generate_counterfactuals(
    X_train_df[closest_idx:closest_idx+1],
    total_CFs=4,
    desired_class="opposite",
    features_to_vary = X_train_df.columns[X_train_df.columns != 'sex'].tolist()
).visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  1.53it/s]

Query instance (original outcome : 0)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,poutcome,y
0,47,management,married,secondary,no,177,yes,no,cellular,14,may,95,3,unknown,0



Diverse Counterfactual set (new outcome: 1)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,poutcome,y
0,-,-,-,-,-,-,-,-,-,-,-,3566,-,-,1
1,-,-,-,-,-,-,-,-,-,-,-,882,-,-,1
2,-,-,-,-,-,-,-,-,telephone,-,-,2876,-,-,1
3,-,-,-,-,-,73788,-,yes,-,-,-,-,-,-,1


In [14]:

exp = dice_ml.Dice(d, m,
                   method="genetic")
start = time.time()
e1_genetic = exp.generate_counterfactuals(
    X_test_df[:100],
    total_CFs=4,
    desired_class="opposite",
    features_to_vary = X_train_df.columns[X_train_df.columns != 'sex'].tolist()
)
end = time.time()
time_genetic = (end - start)/100

100%|██████████| 100/100 [07:06<00:00,  4.26s/it]


In [15]:
exp.generate_counterfactuals(
    X_train_df[closest_idx:closest_idx+1],
    total_CFs=4,
    desired_class="opposite",
    features_to_vary = X_train_df.columns[X_train_df.columns != 'sex'].tolist()
).visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:04<00:00,  4.53s/it]

Query instance (original outcome : 0)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,poutcome,y
0,47,management,married,secondary,no,177,yes,no,cellular,14,may,95,3,unknown,0



Diverse Counterfactual set (new outcome: 1.0)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,poutcome,y
0,18,retired,-,-,-,-3058,-,-,-,8,mar,114,1,-,1
0,45,-,-,tertiary,-,226,-,-,-,15,feb,91,4,success,1
0,28,student,single,-,-,205,no,-,-,-,jul,87,-,other,1
0,29,-,-,-,-,231,no,-,-,30,apr,80,1,-,1


In [16]:

def store_cfs(e1, length):
    list_cfs = []
    for i in range(length):
        list_cfs.append((
            e1.cf_examples_list[i].test_instance_df,
            e1.cf_examples_list[i].final_cfs_df
              ))
    return list_cfs


Note!!! In this case, the model converged to the optimal solution before reaching the maximum number of iterations, but the output probabilities still contain a counterfactual explanation with the same predicted class as the original sample, which is incorrect. The reason is that during the inverse mapping inside the model, the results are rounded to the precision of the original data, and such small changes can lead to significant changes in the predicted values—I have verified this. The reason for the low validation probability here is that there are integer-type categorical variables in the data, but DiCE internally converts them as strings. If you add handle_unknown='ignore', it will not throw an error when encountering a new category, but will set all values to zero (i.e., all one-hot features for that variable are 0, and the model will automatically ignore this new category). Although our integer-type data looks the same as strings to the naked eye, the encoder does not treat them as such, so a large amount of all-zero data represents new categories, which leads to strong prediction bias.

In [17]:
start = time.time()
exp = dice_ml.Dice(d, m,
                   method="gradient")
e1_gradient = exp.generate_counterfactuals(
    X_test_df[0:1],
    total_CFs=4,
    desired_class="opposite",
    features_to_vary=X_train_df.columns[X_train_df.columns != 'sex'].tolist(),
    min_iter=0,
    max_iter=500,
    verbose=True
)



step 1,  loss=3.04252
Diverse Counterfactuals found! total time taken: 00 min 02 sec


In [18]:
for i in range(1, 100):
    e1_i = exp.generate_counterfactuals(
        X_test_df[i:i+1],
        total_CFs=4,
        desired_class="opposite",
        features_to_vary = X_train_df.columns[X_train_df.columns != 'sex'].tolist(),
        min_iter=0,
        max_iter=500
    )
    e1_gradient.cf_examples_list.append(e1_i.cf_examples_list[0])
end = time.time()

time_gradient = (end - start)/100

Diverse Counterfactuals found! total time taken: 00 min 18 sec
Diverse Counterfactuals found! total time taken: 00 min 25 sec
Diverse Counterfactuals found! total time taken: 00 min 18 sec
Diverse Counterfactuals found! total time taken: 00 min 17 sec
Diverse Counterfactuals found! total time taken: 00 min 24 sec
Diverse Counterfactuals found! total time taken: 00 min 06 sec
Diverse Counterfactuals found! total time taken: 00 min 23 sec
Diverse Counterfactuals found! total time taken: 00 min 24 sec
Diverse Counterfactuals found! total time taken: 00 min 24 sec
Diverse Counterfactuals found! total time taken: 00 min 20 sec
Diverse Counterfactuals found! total time taken: 00 min 21 sec
Diverse Counterfactuals found! total time taken: 00 min 24 sec
Diverse Counterfactuals found! total time taken: 00 min 23 sec
Diverse Counterfactuals found! total time taken: 00 min 16 sec
Diverse Counterfactuals found! total time taken: 00 min 18 sec
Diverse Counterfactuals found! total time taken: 00 min

In [19]:
exp.generate_counterfactuals(
    X_train_df[closest_idx:closest_idx+1],
    total_CFs=4,
    desired_class="opposite",
    features_to_vary = X_train_df.columns[X_train_df.columns != 'sex'].tolist(),
    min_iter=0,
    max_iter=500
).visualize_as_dataframe(show_only_changes=True)

Diverse Counterfactuals found! total time taken: 00 min 16 sec
Query instance (original outcome : 0.07599999755620956)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,poutcome,y
0,47.0,management,married,secondary,no,177.0,yes,no,cellular,14.0,may,95.0,3.0,unknown,0.076



Diverse Counterfactual set (new outcome: 1.0)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,poutcome,y
0,-,-,-,-,-,853.0,-,-,-,26.0,mar,1478.0,-,-,1
1,-,-,-,-,-,-1139.0,-,-,-,-,mar,595.0,-,-,1
2,-,-,-,-,-,-,-,-,-,-,-,1913.0,-,-,1
3,-,-,-,-,-,-,-,-,-,-,mar,0.0,-,success,1


In [20]:
cfs_kdtree = store_cfs(e1_kdtree, 100)
cfs_random = store_cfs(e1_random, 100)
cfs_genetic = store_cfs(e1_genetic, 100)
cfs_gradient = store_cfs(e1_gradient, 100)

In [ ]:
from XAI_metrics import calc_valid, calc_sparsity, calc_continuous_proximity, \
    calc_categorical_proximity, calc_manifold_distance, calc_cf_num

valid_kdtree = calc_valid(cfs_kdtree, model, transformer, df.shape[1])
sparsity_kdtree = calc_sparsity(cfs_kdtree, categorical_features)
con_proximity_kdtree = calc_continuous_proximity(cfs_kdtree, numeric_features)
cat_proximity_kdtree = calc_categorical_proximity(cfs_kdtree, categorical_features)
manifold_kdtree = calc_manifold_distance(cfs_kdtree, df, categorical_features)
cf_num_kdtree = calc_cf_num(cfs_kdtree)

valid_random = calc_valid(cfs_random, model, transformer, df.shape[1])
sparsity_random = calc_sparsity(cfs_random, categorical_features)
con_proximity_random = calc_continuous_proximity(cfs_random, numeric_features)
cat_proximity_random = calc_categorical_proximity(cfs_random, categorical_features)
manifold_random = calc_manifold_distance(cfs_random, df, categorical_features)
cf_num_random = calc_cf_num(cfs_random)

valid_genetic = calc_valid(cfs_genetic, model, transformer, df.shape[1])
sparsity_genetic = calc_sparsity(cfs_genetic, categorical_features)
con_proximity_genetic = calc_continuous_proximity(cfs_genetic, numeric_features)
cat_proximity_genetic = calc_categorical_proximity(cfs_genetic, categorical_features)
manifold_genetic = calc_manifold_distance(cfs_genetic, df, categorical_features)
cf_num_genetic = calc_cf_num(cfs_genetic)

valid_gradient = calc_valid(cfs_gradient, model, transformer, df.shape[1])
sparsity_gradient = calc_sparsity(cfs_gradient, categorical_features)
con_proximity_gradient = calc_continuous_proximity(cfs_gradient, numeric_features)
cat_proximity_gradient = calc_categorical_proximity(cfs_gradient, categorical_features)
manifold_gradient = calc_manifold_distance(cfs_gradient, df, categorical_features)
cf_num_gradient = calc_cf_num(cfs_gradient)


In [ ]:
results = {
    "method": ["kdtree", "random", "genetic", "gradient"],
    "Avg Time(s)": [time_kdtree, time_random, time_genetic, time_gradient],
    "Validity": [valid_kdtree, valid_random, valid_genetic, valid_gradient],
    "Sparsity": [sparsity_kdtree, sparsity_random, sparsity_genetic, sparsity_gradient],
    "Proximity_con": [con_proximity_kdtree, con_proximity_random, con_proximity_genetic, con_proximity_gradient],
    "Proximity_cat": [cat_proximity_kdtree, cat_proximity_random, cat_proximity_genetic, cat_proximity_gradient],
    "Manifold": [manifold_kdtree, manifold_random, manifold_genetic, manifold_gradient],
    "Avg CF count": [cf_num_kdtree, cf_num_random, cf_num_genetic, cf_num_gradient]
}

df_result = pd.DataFrame(results)


df_result = df_result.round(2)

In [23]:
df_result

,method,Avg Time(s),Validity,Sparsity,Proximity_con,Proximity_cat,Manifold,Avg CF count
0,kdtree,7.24,1.0,0.66,0.98,0.40,0.00,4.00
1,random,0.56,1.0,0.33,4.33,0.07,618.96,4.00
2,genetic,4.26,1.0,0.58,1.39,0.30,144.16,3.99
3,gradient,20.50,1.0,0.38,1.08,0.17,100.57,4.00


In [24]:
df_result.to_csv('./results/DiCE_result_bank.csv', index=False)